In [1]:
import pandas as pd
import os
import re

In [2]:
DATA_FOLDER_PATH = os.getcwd() + '/../data/KS VERI/KS10/'

In [3]:
FILE_NAMES = ['KS10 SENSOR-1 ISI (Room Sensor).xlsx',
              'KS10 SENSOR-2 ISI (Room Sensor).xlsx']

In [4]:
PATHS = [DATA_FOLDER_PATH + file_name for file_name in FILE_NAMES]

In [5]:
SENSOR_I_DF = pd.read_excel(PATHS[0], sheet_name='KS10 SENSOR-1 ISI (4)')
SENSOR_II_DF = pd.read_excel(PATHS[1], sheet_name='KS10 SENSOR-2 ISI (4)')

In [6]:
def get_INNER_TEMP(sensor_number: int,
                   date_time: str) -> float:
    """
    Get the closest inner temperature value to the given date time.
    
    Args:
        sensor_number: The sensor number.
        date_time: The date time.
    
    Returns:
        The inner temperature value.
    """
    
    if sensor_number > 2 or sensor_number < 1:
        raise ValueError('sensor_number must be 1 or 2.')
    
    # check if the date time is in the correct format with regex
    if not re.match(r'\d{2}\.\d{2}\.\d{4} \d{2}:\d{2}:\d{2}', date_time):
        raise ValueError('date_time must be in the format of dd.mm.yyyy hh:mm:ss')
    
    if sensor_number == 1:
        sensor_df = SENSOR_I_DF
    else:
        sensor_df = SENSOR_II_DF
        
    # convert the date time to datetime object
    date_time = pd.to_datetime(date_time, dayfirst=True)
        
    # find the closest date time
    closest_date_time = min(sensor_df['Date Time'],
                            key=lambda x: abs(x - date_time))
    
    # get the inner temperature value
    inner_temp = sensor_df[sensor_df['Date Time'] == closest_date_time]['KN-2 SENSOR-' + str(sensor_number) + ' ISI'].values[0]
    
    return inner_temp

In [7]:
get_INNER_TEMP(1, '26.08.2020 10:16:00')

27.2532

In [8]:
# save dfs to csv files
SENSOR_I_DF.to_csv(os.getcwd() + '/../data/KS VERI/CLEAN_DATA/KS10_SENSOR_I.csv', index=False)
SENSOR_II_DF.to_csv(os.getcwd() + '/../data/KS VERI/CLEAN_DATA/KS10_SENSOR_II.csv', index=False)